# Анализ лояльности пользователей Яндекс Афиши

- Автор: Попов Егор Валерьевич
- Дата: 07.01.2026

### Цели и задачи проекта

**Цель**: провести исследовательский анализ данных, чтобы понять, какие пользователи с большей вероятностью возвращаются на платформу и делают заказы повторно.

**Задачи:**

1. С помощью SQL-запроса, выгрузить в датафрейм pandas необходимые данные из базы `data-analyst-afisha`, изучить общую информацию о выгруженных данных и оценить корректность выгрузки и объём полученных данных.


2. Провести предобработку данных.

    - привести выручку к единой валюте — российскому рублю;
    - изучить пропущенные значения и при необходимости обработать пропуски;
    - изучить корректность типов данных и при необходимости преобразовать;
    - изучить значения в ключевых столбцах и выявлении ошибок устранить их;
    - проверить данные на явные и неявные дубликаты;
    - изучить значения в ключевых столбцах. Для категоральных столбцов провести нормализацию, а для числовых рассмотреть их распределение и наличие в них выбросов.


3. Создать профиль пользователя по агрегированным признакам, описывающим его поведение. Для каждого пользователя найти:
    - дату первого и последнего заказа;
    - устройство, с которого был сделан первый заказ;
    - регион, в котором был сделан первый заказ;
    - билетного партнёра, к которому обращались при первом заказе;
    - жанр первого посещённого мероприятия (по полю event_type_main);
    - общее количество заказов;
    - среднюю выручку с одного заказа в рублях;
    - среднее количество билетов в заказе;
    - среднее время между заказами. 
    - добавить бинарные признаки: `is_two` — совершил ли пользователь 2 и более заказа, `is_five` — совершил ли пользователь 5 и более заказов.

   Для оценки репрезентативности сформированных профилей пользователей и проверки их на аномалые значения необходимо рассчитать:
   
    - общее число пользователей в выборке;
    - среднюю выручку с одного заказа;
    - долю пользователей, совершивших 2 и более заказа;
    - долю пользователей, совершивших 5 и более заказов;
    - статистические показатели (по общему числу заказов; по среднему числу билетов в заказе; по среднему количеству дней между покупками).
    
    
    
4. Провести исследовательский анализ данных:
    - 4.1. Исследовать признаки первого заказа и их связи с возвращением на платформу:
        - изучить распределение пользователей по различным признакам. Для этого необходимо сгрупировать пользователей по сегментам и расчитать общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака;
        - проанализировать возвраты пользователей. Для каждого сегмента вычислить долю пользователей, совершивших два и более заказа;
        - проверить продуктовые гипотезы (формулировки гипотез будут приведены в разделе посвященном ИАД).
    - 4.2. Исследовать поведение пользователей через показатели выручки и состава заказа:
        - изучить количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают;
        - проследить связь между средней выручкой сервиса с заказа и повторными заказами;
        - сравнить распределение по средней выручке с заказа в двух группах пользователей: совершившие 2–4 заказа и совершившие 5 и более заказов;
        - проанализировать влияние среднего количества билетов в заказе на вероятность повторной покупки;
    - 4.3. Исследовать временные характеристики первого заказа и их влияния на повторные покупки:
        - изучить временные параметры, связанные с первым заказом пользователей;
        - проанализировать, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей;
        - изучить, как средний интервал между заказами влияет на удержание клиентов.
    - 4.4. Провести корреляционный анализ количества покупок и признаков пользователя.
    
5. Сформулировать общие выводы и рекомендации по проведённому анализу.

### Описание данных

Необходимые для анализа данные сервиса **Яндекс Афиша** представлены в базе данных PostgreSQL `data-analyst-afisha`. Подключение к БД осуществлялось с помощью программы **DBeaver**.

Схема `afisha` базы данных `data-analyst-afisha` содержит пять таблиц.

Таблица `purchases` содержит информацию о заказах билетов:

- `order_id` — идентификатор заказа;  
- `user_id` — идентификатор пользователя, совершившего заказ;  
- `created_dt_msk` — дата создания заказа по московскому времени;  
- `created_ts_msk` — дата и время создания заказа по московскому времени;  
- `event_id` — идентификатор мероприятия из таблицы `events`;  
- `cinema_circuit` — сеть кинотеатров, где проходит мероприятие. Если оно проходит не в кинотеатре, то здесь будет значение нет;  
- `age_limit` — возрастное ограничение мероприятия;  
- `currency_code` — валюта оплаты;  
- `device_type_canonical` — тип устройства, с которого был оформлен заказ (например, `mobile` — мобильные устройства, `desktop` — компьютеры);  
- `revenue` — выручка от заказа;  
- `service_name` — название билетного оператора;  
- `tickets_count` — количество купленных билетов;  
- `total` — общая сумма заказа.

Таблица `events` содержит данные о мероприятиях, доступных на платформе:

- `event_id` — идентификатор мероприятия.  
- `event_name_code` — название мероприятия в закодированном виде.  
- `event_type_description` — описание мероприятия.  
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее)  
- `organizers` — организаторы мероприятия.  
- `city_id` — идентификатор города проведения мероприятия из таблицы `cities`.  
- `venue_id` — идентификатор площадки проведения мероприятия из таблицы `venues`.

Таблица `venues` содержит сведения о площадках проведения мероприятий:

- `venue_id` — идентификатор площадки;  
- `venue_name` — название площадки;  
- `address` — адрес площадки.

Таблица `city` содержит список городов, относящихся к регионам:

- `city_id` — идентификатор города;  
- `city_name` — название города;  
- `region_id` — идентификатор региона, в котором расположен город, из таблицы `regions`.

Таблица `regions` содержит список регионов, в которых проводятся мероприятия:

- `region_id` — идентификатор региона;  
- `region_name` — название региона.


### Содержимое проекта

<font color='#777778'>Перечислите основные шаги проекта или напишите оглавление. Используйте описание проекта, чтобы зафиксировать основные шаги.</font>

---

<a id="toc"></a>
- [1. Загрузка данных и знакомство с ними](#1-bullet)
- [2. Предобработка данных](#2-bullet)
- [3. Исследовательский анализ данных](#3-bullet)
- [4. Итоговый вывод и рекомендации](#4-bullet)

<a id="1-bullet"></a>
## 1. Загрузка данных и знакомство с ними

<font color='#777778'>Задача 1.1: Напишите SQL-запрос, выгружающий в датафрейм pandas необходимые данные. Используйте следующие параметры для подключения к базе данных `data-analyst-afisha`: </font>

- описать параметры подключения
- Для выгрузки используйте запрос, который вы писали в третьем уроке проекта, «Выгрузка данных с помощью SQL». Инструкция по тому, как это сделать, дана в четвёртом уроке, «Подключение к базе данных с помощью Python».
</font>

<font color='#777778'>Задача 1.2: Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных. </font>

---

### Промежуточный вывод

<font color='#777778'>Сделайте промежуточный вывод о полученных данных: данные какого объёма вам предоставили, соответствуют ли данные описанию, встречаются ли в них пропуски, используются ли верные типы данных. Отметьте другие особенности данных, которые вы обнаружите на этой стадии и на которые стоит обратить внимание при предобработке. </font>
**Особенности данных**:
